## Web scraping exercise

Define a generic function `SOS_help` which retrieves help results from Stack Overflow Stunning results. <br>

The following command works just fine:

In [2]:
import numpy as np
import pandas as pd
import requests
import traceback
np.random.uniform(-1, 1, size=100)

array([ 0.51407705, -0.34088564, -0.62698977,  0.4280116 ,  0.33293041,
        0.22675656,  0.86827163, -0.70202618,  0.38388976,  0.41206605,
        0.80718013, -0.9836492 , -0.61759477,  0.93984917,  0.28034366,
       -0.08820831,  0.67527228,  0.12616929,  0.73805339, -0.57411595,
        0.03928805, -0.88246993, -0.48825811, -0.21970186, -0.31126985,
       -0.6143659 , -0.39094558, -0.95302047,  0.63464084,  0.66820147,
       -0.97804453, -0.90687263, -0.84760986,  0.40659792, -0.55029817,
       -0.1318928 ,  0.78411697,  0.70129394, -0.67522022, -0.07255266,
       -0.84961242, -0.41114681, -0.18971322, -0.65650331, -0.10703908,
       -0.83399105, -0.65246297, -0.97001178, -0.20090462,  0.40521938,
       -0.78715625, -0.39177116, -0.67493672,  0.68694773, -0.21279524,
       -0.02893788,  0.71687035, -0.04578704, -0.66273543,  0.2633314 ,
       -0.19642678,  0.20024432, -0.88517658,  0.44590563, -0.11109673,
       -0.14027343, -0.63554851,  0.06979609, -0.49573246,  0.50

`print_out` prints command execution

In [3]:
def print_output(command):
    command = 'print(' + command + ')'
    return exec(command)

In [4]:
print_output(command = 'np.random.uniform(-1, 1, size=100)')

[-0.13765871  0.02888692 -0.38533686  0.11042077  0.94395697 -0.9507452
  0.92961457  0.67887469  0.90492067  0.7554485   0.02989511  0.47241788
  0.35017362  0.20228004 -0.22548429 -0.72538797 -0.89153595  0.54309199
 -0.86182511 -0.84572348 -0.18809935  0.84941057 -0.86426795  0.33447086
 -0.70641556 -0.36564351 -0.92259416  0.09975389  0.7132764   0.20741772
  0.47029686  0.17331952  0.58507806 -0.59571319 -0.58040145  0.85577706
 -0.31621782  0.17065652 -0.3037388  -0.16547727 -0.50028429 -0.16517597
 -0.78881881  0.44968497 -0.18152642 -0.02931142  0.56973686 -0.10945861
 -0.70897733 -0.19864681 -0.32227465  0.98866022 -0.83953675  0.03337301
 -0.54145946 -0.79117459 -0.43450108  0.25210664  0.09060075 -0.53531034
 -0.89343434 -0.30927609  0.29317935 -0.60960937  0.30315684 -0.53299717
  0.51085781 -0.84476471  0.35073839  0.9315844  -0.45507741 -0.12726035
 -0.18958066 -0.5419927  -0.40213565  0.39944341 -0.17503028 -0.2606458
 -0.58447645  0.53942862  0.50233918 -0.98114319  0.4

if `print_out` results in an error, this is printed

In [5]:
print_output(command = 'nn.random.uniform(-1, 1, size=100)')

NameError: name 'nn' is not defined

In [6]:
print_output(command = 'np.random.uniform(-1, 1, siz=100)')

TypeError: uniform() got an unexpected keyword argument 'siz'

Create a function `get_SOS_help` which: <br>
    - Prints "works as intended" if no error. <br>
    - Prints the first link from stack overflow related to the error. As an example: <br>
        `print_output(command = 'np.random.uniform(-1, 1, siz=100)'`
        should retrieve the following link:
        https://stackoverflow.com/questions/72537485/typeerror-uniform-got-an-unexpected-keyword-argument-low-size <br>
    - Prints the most voted help
    - Opens a new browser using the link

## Create a malfunctioning code and use this function on it

In [7]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import traceback
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Cargar la clave de API desde el archivo .env
load_dotenv()
api_key = os.getenv("OPENAI_KEY")\


# Configurar el cliente de OpenAI
client = OpenAI(api_key=api_key)

def get_SOS_help(error_message):
    url = f"https://www.google.com/search?q={error_message} site:stackoverflow.com"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException as e:
        print("❌ Error al conectar con Google:", e)
        return None
    
    allData = soup.find_all("div", {"class": "g"})
    urls = []
    
    for data in allData:
        link_tag = data.find("a")
        if link_tag and link_tag.get('href'):
            urls.append(link_tag['href'])
    
    if not urls:
        print("❌ No se encontraron resultados en StackOverflow")
        return None
    
    print("🔍 Los mejores resultados de StackOverflow son:")
    print(f"👉 {urls[0]}")
    
    try:
        response = requests.get(urls[0], headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException as e:
        print("❌ Error al conectar con StackOverflow:", e)
        return None

    votes = soup.find_all("div", {"class": "js-vote-count"})
    comments = soup.find_all("div", {"class": "s-prose js-post-body"})
    
    votes_list = [int(vote.get_text().strip()) for vote in votes if vote.get_text().strip().isdigit()]
    comments_list = [comment.get_text().strip() for comment in comments]
    
    if not votes_list or not comments_list:
        print("⚠️ No se pudieron extraer comentarios o votos de la respuesta.")
        return None
    
    data = list(zip(votes_list, comments_list))
    df = pd.DataFrame(data, columns=['Votes', 'Comment'])
    df = df.sort_values(by='Votes', ascending=False)
    
    top_comment = df.Comment.iloc[0]
    print("\n🎉 ¡Aquí tienes el comentario más votado! 🎉\n")
    
    return top_comment

def get_AI_help(error_message):
    # Construir el prompt con instrucciones específicas
    prompt = (
        f"Eres un asistente que ayuda a programadores con errores de Python. "
        f"Por favor, proporciona una explicación detallada y una solución para el error: '{error_message}'. "
        "Usa el siguiente formato:\n\n"
        "⚠️ **Error encontrado:** [Explicación del error]\n\n"
        "💡 **Posible solución:** [Solución propuesta con ejemplo de código]\n\n"
        "Incluye ejemplos de código y traceback cuando sea relevante."
    )

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.7
        )
        
        # Extraer y devolver la respuesta formateada de la IA
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error al generar el comentario con OpenAI: {e}")
        return None

def execute_code_with_help(command):
    try:
        exec(command)
    except Exception as e:
        error_message = str(e)
        full_traceback = traceback.format_exc()
        print(f"⚠️ Error encontrado: {error_message}")
        
        # Buscar ayuda en StackOverflow
        print("🔍 Buscando solución en StackOverflow...")
        solution = get_SOS_help(error_message)

        if solution:
            print("💡 Posible solución encontrada en StackOverflow:")
            print(solution)
        else:
            print("❌ No se encontró una solución relevante en StackOverflow.")
        
        # Consultar a OpenAI para obtener una respuesta formateada
        print("\n🤖 Consultando a GPT-3.5 para obtener ayuda...\n")
        ai_solution = get_AI_help(error_message)

        if ai_solution:
            print(ai_solution)
        else:
            print("❌ No se pudo obtener una respuesta de GPT-3.5.")


In [8]:
execute_code_with_help('int("hola")')


⚠️ Error encontrado: invalid literal for int() with base 10: 'hola'
🔍 Buscando solución en StackOverflow...
🔍 Los mejores resultados de StackOverflow son:
👉 https://stackoverflow.com/questions/1841565/valueerror-invalid-literal-for-int-with-base-10

🎉 ¡Aquí tienes el comentario más votado! 🎉

💡 Posible solución encontrada en StackOverflow:
The error message means that the string provided to int could not be parsed as an integer. The part at the end, after the :, shows the string that was provided.
In the case described in the question, the input was an empty string, written as ''.
Here is another example - a string that represents a floating-point value cannot be converted directly with int:
>>> int('55063.000000')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ValueError: invalid literal for int() with base 10: '55063.000000'

Instead, convert to float first:
>>> int(float('55063.000000'))
55063

🤖 Consultando a GPT-3.5 para obtener ayuda...

⚠️ **Error encon